In [1]:
!pip install -q polars


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import polars as pl
from datetime import datetime
from util import *

# Downlaod CSV

Protocol

In [3]:
for d in files:
    df = pl.read_csv(f"df_protocol/{d}").sort("Timestamp")
    df = cast_datetime(df, col="Timestamp")
    df = truncate_datetime(df, col="Timestamp",trun="1d")

    temp = df["Timestamp"].value_counts().sort("Timestamp").filter(pl.col("count") > 1)
    if temp.shape[0]:
        print(d, "has duplicates")
        print(temp)

    df = df.unique("Timestamp", keep="last")
    min_date.append(df["Timestamp"].dt.min())
    max_date.append(df["Timestamp"].dt.max())


min_date = pl.Series("min_date", min_date).dt.min()
max_date = pl.Series("max_date", max_date).dt.max()

print(min_date, max_date)

result_raw = get_empty_result(
    min_date, max_date, step="day", col="Timestamp")

2018-04-27 00:00:00 2024-08-25 00:00:00


In [5]:
STEP = 1000
os.makedirs(f"result{STEP}", exist_ok=True)

for i in range(0, len(files), STEP):
    result = result_raw
    for d in files[i:i+STEP]:
        df = pl.read_csv(f"df_protocol/{d}").sort("Timestamp")
        df = df.drop(["Date"])
        df = cast_column_type_without_date(df)
        df = cast_datetime(df, has_ms=False, col="Timestamp")
        df = truncate_datetime(df, "Timestamp", "1d")
        result = result.join(df, on="Timestamp", how="left")

    result = drop_null_rows(result)
    result = result.sort("Timestamp")
    result.write_csv(f"result{STEP}/result{i}.csv")

Chain

In [2]:
min_date = []
max_date = []
files = os.listdir("df_chain")


for d in files:
    try:
        df = pl.read_csv(f"df_chain/{d}").sort("date")
        df = cast_datetime(df, col="date", dt_format="%Y-%m-%d")
        # df = truncate_datetime(df, col="date",trun="1d")

        temp = df["date"].value_counts().sort("date").filter(pl.col("count") > 1)
        if temp.shape[0]:
            print(d, "has duplicates")
            print(temp)

        df = df.unique("date", keep="last")
        min_date.append(df["date"].dt.min())
        max_date.append(df["date"].dt.max())
    except:
        print(d)


min_date = pl.Series("min_date", min_date).dt.min()
max_date = pl.Series("max_date", max_date).dt.max()

print(min_date, max_date)

result_raw = get_empty_result(
    min_date, max_date, step="day", col="date")

2018-04-27 00:00:00 2024-08-25 00:00:00


In [3]:
STEP = 500
os.makedirs(f"result_chain{STEP}", exist_ok=True)

for i in range(0, len(files), STEP):
    result = result_raw
    for d in files[i:i+STEP]:
        df = pl.read_csv(f"df_chain/{d}").sort("date")
        df = cast_column_type_without_date(df)
        df = cast_datetime(df, dt_format="%Y-%m-%d", col="date")
        df = truncate_datetime(df, "date", "1d")
        result = result.join(df, on="date", how="left")

    result = drop_null_rows(result)
    result = result.sort("date")
    result.write_csv(f"result_chain{STEP}/result{i}.csv")